In [1]:
from gwpy.timeseries import TimeSeries
from gwpy.time import to_gps
import numpy as np

In [2]:
t1 = "2017-08-17T12:40:54" 
t2 = "2017-08-17T12:41:10"
detector = "H1"

In [3]:
t1 = to_gps(t1)
t2 = to_gps(t2)
t0 = (t1 + t2) / 2

In [4]:
strain = TimeSeries.fetch_open_data(detector, t1, t2, cache=True)

In [5]:
psd = strain.psd()
f0 = psd.f0.value
df = psd.df.value
f = np.arange(f0, f0 + df * len(psd), df)

#-- constants
# speed of light:
clight = 2.99792458e8                # m/s
# Newton's gravitational constant
G = 6.67259e-11                      # m^3/kg/s^2 
# one parsec, popular unit of astronomical distance (around 3.26 light years)
parsec = 3.08568025e16               # m
# solar mass
MSol = 1.989e30                      # kg
# solar mass in seconds (isn't relativity fun?):
tSol = MSol*G/np.power(clight,3)     # s
# Single-detector SNR for detection above noise background: 
SNRdet = 8.
# conversion from maximum range (horizon) to average range:
Favg = 2.2648
# mass of a typical neutron star, in solar masses:
mNS = 1.4

# Masses in solar masses
m1 = m2 = mNS    
mtot = m1+m2  # the total mass
eta = (m1*m2)/mtot**2  # the symmetric mass ratio
mchirp = mtot*eta**(3./5.)  # the chirp mass (FINDCHIRP, following Eqn 3.1b)

# distance to a fiducial BNS source:
dist = 1.0                           # in Mpc
Dist =  dist * 1.0e6 * parsec /clight # from Mpc to seconds

# We integrate the signal up to the frequency of the "Innermost stable circular orbit (ISCO)" 
R_isco = 6.      # Orbital separation at ISCO, in geometric units. 6M for PN ISCO; 2.8M for EOB 
# frequency at ISCO (end the chirp here; the merger and ringdown follow) 
f_isco = 1./(np.power(R_isco,1.5)*np.pi*tSol*mtot)
# minimum frequency (below which, detector noise is too high to register any signal):
f_min = 20. # Hz
# select the range of frequencies between f_min and fisco
fr = np.nonzero(np.logical_and(f > f_min , f < f_isco))
# get the frequency and spectrum in that range:
ffr = f[fr]

# In stationary phase approx, this is htilde(f):  
# See FINDCHIRP Eqns 3.4, or 8.4-8.5 
htilde = (2.*tSol/Dist)*np.power(mchirp,5./6.)*np.sqrt(5./96./np.pi)*(np.pi*tSol)
htilde *= np.power(np.pi*tSol*ffr,-7./6.)
htilda2 = htilde**2

sspec = psd.value
sspecfr = sspec[fr]
# compute "inspiral horizon distance" for optimally oriented binary; FINDCHIRP Eqn D2:
D_BNS = np.sqrt(4.*np.sum(htilda2/sspecfr)*df)/SNRdet
# and the "inspiral range", averaged over source direction and orientation:
R_BNS = D_BNS/Favg

In [6]:
bns_horizon = D_BNS
bns_range = R_BNS